## Dubali Local Explorer AI Chatbot

In [2]:
# Import the libraries
import os, json, ast
import pandas as pd
import openai
from openai import OpenAI

In [3]:
#Let's check the version of OpenAI
import openai
print(openai.__version__)

1.95.1


In [4]:
# Read key from text file
with open("openai_key.txt", "r") as f:
    api_key = f.read().strip()

In [5]:
# Pass key to OpenAI client
client = OpenAI(api_key=api_key)

In [6]:
# Read the dataset and read the Laptop Dataset
dubai_tourist_spots = pd.read_csv('dubai_tours.csv')
dubai_tourist_spots.head()

,name,tags,min_budget,max_budget,duration_hours,suitable_for,description
0,Desert Safari,"adventure,outdoor,desert",300,700,6.0,"family,couple,friends","Dune‑bashing, camel rides, traditional dinner ..."
1,JBR Beach Day,"beach,outdoor,relaxation",0,100,5.0,"family,couple,friends",Relax on sands and enjoy water sports at JBR B...
2,Motiongate Dubai,"theme‑park,Hollywood,rides",150,350,6.0,"family,friends",Hollywood‑inspired rides and shows at Dubai Pa...
3,Dubai Museum Tour,"culture,museum,historical",50,150,2.0,"family,couple,solo",Explore Al Fahidi Fort history exhibitions.
4,Dubai Aquarium & Underwater Zoo,"marine,indoor,family",100,250,3.0,"family,couple,solo","See sharks, rays, and marine life within Dubai..."


### 1. Initialize Conversation for Dubali Local Explorer AI Chatbot

In [8]:
def initialize_conversation():
    """
    Returns a rich initial prompt including:
    - A system message defining assistant personality and constraints
    - Multiple few-shot examples that demonstrate reasoning
    - Sets context for consistent user interaction
    """

    initial_system_prompt = {
        "role": "system",
        "content": (
            "You are DubaiLocalExplorer, an intelligent assistant that helps tourists explore Dubai. "
            "You engage in natural conversation to understand user preferences, confirm intent, and suggest "
            "personalized day-trip itineraries. Use friendly, concise language. "
            "\n\nYour goal is to:\n"
            "- Help users clarify their interests, budget, group type, and duration\n"
            "- Ask clarifying questions if data is missing\n"
            "- Suggest experiences that match those constraints\n"
            "- Think step-by-step and summarize total cost & time\n\n"
            "Always tailor suggestions based on their profile. Keep the tone warm and adaptive."
        )
    }

    dubai_experiences_examples = [
        {
            "role": "user",
            "content": "Hi! I'm in Dubai for 1 day. I'd love to explore cultural sites and try local food."
        },
        {
            "role": "assistant",
            "content": (
                "That sounds like a wonderful day! Here's a quick plan:\n"
                "1. Start at Al Fahidi Historical District to explore Dubai Museum.\n"
                "2. Ride an abra across the Creek to the Gold and Spice Souks.\n"
                "3. Try Emirati cuisine at Al Seef or Arabian Tea House.\n\n"
                "Would you prefer walking between places, or prefer transport arranged?"
            )
        },
        {
            "role": "user",
            "content": "I'm traveling with my wife, and we have around 700 AED to spend."
        },
        {
            "role": "assistant",
            "content": (
                "Thanks for sharing! That gives us good flexibility.\n"
                "I'll make sure all experiences are within budget for a couple. "
                "Would you like to include modern attractions like Dubai Frame or Museum of the Future?"
            )
        },
        {
            "role": "user",
            "content": "We are a group of friends, want nightlife and beaches, budget is tight though."
        },
        {
            "role": "assistant",
            "content": (
                "Got it — you're a group looking for beach and nightlife fun, but mindful of costs.\n"
                "Let me find beach clubs or night spots with free or low entry fees, and suggest group-friendly spots. "
                "Can you tell me how many days you’re staying?"
            )
        },
        {
            "role": "user",
            "content": "Just one full day. Don’t want to waste time in malls."
        },
        {
            "role": "assistant",
            "content": (
                "Understood — one focused day without shopping distractions.\n"
                "How about this:\n"
                "- Morning: Relax at Kite Beach (free entry, lots of cafés)\n"
                "- Afternoon: Explore JBR beachfront and walk The Beach boardwalk\n"
                "- Evening: Rooftop dinner at a budget-friendly lounge near Dubai Marina\n\n"
                "Want me to include transport suggestions?"
            )
        }
    ]

    return [initial_system_prompt] + dubai_experiences_examples


In [9]:
def get_chat_model_completions(conversation, model="gpt-3.5-turbo", temperature=0.4, print_response=True):
    """
    Submits a message list (conversation) to the OpenAI ChatCompletion API and prints the assistant's response.
    
    Args:
        conversation (list): List of dicts like [{"role": "user", "content": "..."}, ...]
        model (str): LLM model to use (default: "gpt-3.5-turbo")
        temperature (float): Sampling temperature
        print_response (bool): Whether to print response to console

    Returns:
        str: Assistant's message content
    """

    # Validate input
    if not isinstance(conversation, list) or not all("role" in m and "content" in m for m in conversation):
        raise ValueError("Conversation must be a list of dicts with 'role' and 'content' keys")

    try:
        # Make API call
        response = client.chat.completions.create(
            model=model,
            messages=conversation,
            temperature=temperature
        )

        message = response.choices[0].message.content.strip()

        if print_response:
            print("\n Assistant Response:\n")
            print(message)
        
        return message

    except openai.BadRequestError as e:
        print("Invalid request (400):", e)
    except openai.AuthenticationError as e:
        print("Authentication failed (401):", e)
    except openai.PermissionDeniedError as e:
        print("Permission denied (403):", e)
    except openai.NotFoundError as e:
        print("Not found (404):", e)
    except openai.UnprocessableEntityError as e:
        print("Unprocessable entity (422):", e)
    except openai.RateLimitError as e:
        print("Rate limited (429):", e)
    except openai.InternalServerError as e:
        print("Server error (5xx):", e)
    except openai.APIConnectionError as e:
        print("Connection issue:", e)
    except openai.APIStatusError as e:
        print(f"Error status {e.status_code}:", e.response)
    except openai.APIError as e:
        print("General API error:", e)
    except Exception as e:
        print("Unexpected error:", e)


In [10]:
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What are some fun things to do in Dubai?"}
]

get_chat_model_completions(conversation)


 Assistant Response:

There are plenty of fun activities to do in Dubai! Here are some popular options:

1. Visit the Burj Khalifa: Enjoy breathtaking views of the city from the world's tallest building.
2. Explore the Dubai Mall: Shop, dine, and visit attractions like the Dubai Aquarium and Underwater Zoo.
3. Desert Safari: Experience a thrilling desert adventure with activities like dune bashing, camel riding, and sandboarding.
4. Dubai Marina: Take a leisurely stroll along the waterfront, enjoy a boat ride, or dine at one of the many restaurants.
5. Dubai Fountain Show: Watch the mesmerizing fountain show set to music outside the Dubai Mall.
6. Ski Dubai: Experience snow in the middle of the desert at this indoor ski resort.
7. Dubai Parks and Resorts: Enjoy theme parks like Motiongate, Bollywood Parks, and Legoland.
8. Visit the Dubai Miracle Garden: Explore the world's largest natural flower garden with unique floral displays.
9. Dubai Frame: Get a unique perspective of the city 

"There are plenty of fun activities to do in Dubai! Here are some popular options:\n\n1. Visit the Burj Khalifa: Enjoy breathtaking views of the city from the world's tallest building.\n2. Explore the Dubai Mall: Shop, dine, and visit attractions like the Dubai Aquarium and Underwater Zoo.\n3. Desert Safari: Experience a thrilling desert adventure with activities like dune bashing, camel riding, and sandboarding.\n4. Dubai Marina: Take a leisurely stroll along the waterfront, enjoy a boat ride, or dine at one of the many restaurants.\n5. Dubai Fountain Show: Watch the mesmerizing fountain show set to music outside the Dubai Mall.\n6. Ski Dubai: Experience snow in the middle of the desert at this indoor ski resort.\n7. Dubai Parks and Resorts: Enjoy theme parks like Motiongate, Bollywood Parks, and Legoland.\n8. Visit the Dubai Miracle Garden: Explore the world's largest natural flower garden with unique floral displays.\n9. Dubai Frame: Get a unique perspective of the city from this ar

In [11]:
conversation = [
    {"role": "system", "content": "You are a travel assistant."},
    {"role": "user", "content": "We're couple visiting Dubai for 2 days. Love culture and food."},
    {"role": "user", "content": "We want to keep it under 600 AED."}
]

get_chat_model_completions(conversation)


 Assistant Response:

Great! With a budget of 600 AED for 2 days in Dubai, here is a suggested itinerary that includes cultural experiences and delicious food:

Day 1:
1. Morning: Start your day with a visit to the Dubai Museum in Al Fahidi Fort to learn about the history and culture of Dubai. Admission fee is only 3 AED per person.
2. Lunch: Head to the Al Ustad Special Kebab in Bur Dubai for a delicious and affordable meal of traditional Persian kebabs. Expect to spend around 50 AED for two.
3. Afternoon: Explore the Al Fahidi Historical Neighbourhood, also known as Al Bastakiya, with its narrow lanes, wind-towers, and art galleries. Entrance is free.
4. Evening: Visit the Dubai Spice Souk and Gold Souk in Deira to experience the vibrant atmosphere and shop for spices and gold. You can also take a traditional abra (water taxi) ride across Dubai Creek for just 1 AED.
5. Dinner: Enjoy a budget-friendly meal at Ravi Restaurant in Satwa, known for its delicious Pakistani and Indian dish

"Great! With a budget of 600 AED for 2 days in Dubai, here is a suggested itinerary that includes cultural experiences and delicious food:\n\nDay 1:\n1. Morning: Start your day with a visit to the Dubai Museum in Al Fahidi Fort to learn about the history and culture of Dubai. Admission fee is only 3 AED per person.\n2. Lunch: Head to the Al Ustad Special Kebab in Bur Dubai for a delicious and affordable meal of traditional Persian kebabs. Expect to spend around 50 AED for two.\n3. Afternoon: Explore the Al Fahidi Historical Neighbourhood, also known as Al Bastakiya, with its narrow lanes, wind-towers, and art galleries. Entrance is free.\n4. Evening: Visit the Dubai Spice Souk and Gold Souk in Deira to experience the vibrant atmosphere and shop for spices and gold. You can also take a traditional abra (water taxi) ride across Dubai Creek for just 1 AED.\n5. Dinner: Enjoy a budget-friendly meal at Ravi Restaurant in Satwa, known for its delicious Pakistani and Indian dishes. Plan to spe

In [12]:
conversation = [
    {"role": "system", "content": "You extract user travel preferences into JSON format."},
    {"role": "user", "content": "I'm coming with kids, want beach and outdoor stuff. Budget is around 700 AED."},
    {"role": "user", "content": "Staying for one day only."},
    {"role": "user", "content": (
        "Please extract my travel preferences as structured JSON with keys: "
        "'interests', 'duration_days', 'budget_aed', 'group_type'."
    )}
]

response_from_assistant  = get_chat_model_completions(conversation)


 Assistant Response:

{
  "interests": ["beach", "outdoor activities"],
  "duration_days": 1,
  "budget_aed": 700,
  "group_type": "family with kids"
}


In [13]:
debug_user_input = 'my max budget is 1000 AED'

In [14]:
conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(conversation)
print(debug_response_assistant)


 Assistant Response:

{
  "interests": ["beach", "outdoor activities"],
  "duration_days": 1,
  "budget_aed": 1000,
  "group_type": "family with kids"
}
{
  "interests": ["beach", "outdoor activities"],
  "duration_days": 1,
  "budget_aed": 1000,
  "group_type": "family with kids"
}


In [15]:
def moderation_check(user_input):
    """
    Uses OpenAI's Moderation API to check if input violates content policy.
    Returns a dict with moderation status and flagged categories.
    """
    try:
        response = client.moderations.create(input=user_input)
        result = response.results[0]

        if result.flagged:
            return f"Flagged by moderation. Categories: {result.categories}"
        
        return "Moderation check passed for given input: "+ user_input

    except Exception as e:
        print("Exception occurred:", e)
        return {
            "flagged": False,
            "categories": {},
            "error": str(e)
        }


In [16]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Moderation check passed for given input: my max budget is 1000 AED


In [17]:
print(moderation_check("I want to kill Ravan."))
print(moderation_check("Need a Dubai Desert Safari Package"))

Flagged by moderation. Categories: Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False)
Moderation check passed for given input: Need a Dubai Desert Safari Package


**Let's now check moderation on the assistant's response debug_response_assistant.**

In [19]:
moderation_check(debug_response_assistant)

'Moderation check passed for given input: {\n  "interests": ["beach", "outdoor activities"],\n  "duration_days": 1,\n  "budget_aed": 1000,\n  "group_type": "family with kids"\n}'

### 2. Intent & Profile Confirmation Layer

In [21]:
intent_extraction_prompt = [
    {"role": "system", "content": (
        "You are a helpful assistant that extracts travel intent as structured JSON. "
        "Your goal is to return a user profile with four keys: 'interests', 'duration_days', 'budget_aed', 'group_type'. "
        "If any values are missing, return them under a second key called 'missing_info'. "
        "Always think step-by-step before creating the JSON."
    )},
    {"role": "user", "content": "We're couple want to visit some cultural places and markets in Dubai & trip is for 5 days with a budget of 500 AED to spend."},
    {"role": "assistant", "content": (
        "{\n"
        "  'interests': ['culture', 'shopping'],\n"
        "  'budget_aed': 500,\n"
        "  'duration_days': 5,\n"
        "  'group_type': 'couple',\n"
        "}"
    )},
    
    {"role": "user", "content": "I want to visit some cultural places and markets in Dubai. I have about 500 AED to spend."},
    {"role": "assistant", "content": (
        "Step-by-step reasoning:\n"
        "- The user mentions cultural places and markets → interests = ['culture', 'shopping']\n"
        "- They mention a budget of 500 AED → budget_aed = 500\n"
        "- Duration is unspecified → missing_info = ['duration_days']\n"
        "- Group type not mentioned → missing_info += ['group_type']\n"
        "{\n"
        "  'interests': ['culture', 'shopping'],\n"
        "  'budget_aed': 500,\n"
        "  'missing_info': ['duration_days', 'group_type']\n"
        "}"
    )},

    {"role": "user", "content": "We are a couple traveling for 2 days, looking for adventure and beach activities."},
    {"role": "assistant", "content": (
        "Step-by-step reasoning:\n"
        "- Interests: adventure and beach → interests = ['adventure', 'beach']\n"
        "- Duration: 2 days\n"
        "- Group type: couple\n"
        "- Budget not mentioned → missing_info = ['budget_aed']\n"
        "{\n"
        "  'interests': ['adventure', 'beach'],\n"
        "  'duration_days': 2,\n"
        "  'group_type': 'couple',\n"
        "  'missing_info': ['budget_aed']\n"
        "}"
    )},
]


In [22]:
def intent_confirmation_layer(conversation):
    """
    Extracts the user profile (interests, duration, budget, group_type) 
    using few-shot prompting and returns missing fields if needed.
    """

    # Add few-shot prompt history
    prompt = intent_extraction_prompt + [conversation] + [{
        "role": "user",
        "content": (
            "Please extract my travel preferences as structured JSON. "
            "Use the format shown earlier with step-by-step reasoning."
        )
    }]

    # LLM Call
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=prompt,
        temperature=0.4
    )

    result_text = response.choices[0].message.content.strip()


    # Attempt to evaluate the returned JSON safely
    import json,re
    def extract_json_from_text(result_text):
        try:
            # Match the first well-formed {...} block using regex
            match = re.search(r'\{[\s\S]*?\}', result_text)
            if match:
                json_candidate = match.group(0)
                profile = json.loads(json_candidate)
                return profile
            else:
                raise ValueError("No JSON object found.")
        except Exception as e:
            return {
                "error": "Failed to parse LLM output.",
                "raw": result_text,
                "details": str(e)
            }

    result_text = result_text.replace("'", '"')

    profile = extract_json_from_text(result_text)

    return profile

In [23]:
input_conversation = {"role": "user", "content": "We're couple visiting Dubai for 2 days, we love culture and food & we want to keep it under 600 AED."}


In [24]:
intent_confirmation_response = intent_confirmation_layer(input_conversation)
display(intent_confirmation_response)

{'interests': ['culture', 'food'],
 'duration_days': 2,
 'budget_aed': 600,
 'group_type': 'couple'}

In [25]:
def dictionary_present(response):
    """
    Sends the user request to the OpenAI Completion API with few-shot examples,
    expecting a valid dictionary in response.

    Returns:
        dict if dictionary is valid, otherwise None
    """

    # Few-shot examples to guide the model
    few_shot_prompt = """
            You are an assistant that extracts structured travel intent in Python dictionary format.
            
            Example 1:
            User: I want to explore historical sites and shop for souvenirs. I'm coming with my family for 3 days and want to spend about 800 AED.
            Response:
            {
              "interests": ["culture", "shopping"],
              "duration_days": 3,
              "budget_aed": 800,
              "group_type": "family"
            }
            
            Example 2:
            User: We are a couple interested in beaches and relaxing spots. We’ll be here just one day and can spend around 500 AED.
            Response:
            {
              "interests": ["beach", "relaxation"],
              "duration_days": 1,
              "budget_aed": 500,
              "group_type": "couple"
            }
            """

    messages = [{"role": "system", "content":few_shot_prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_model_completions(messages)

    return confirmation


In [26]:
#user_request = "I'm visiting with friends, want to party and enjoy nightlife. Staying for 2 days with a budget of 600 AED."
profile_dict = dictionary_present(intent_confirmation_response)

if profile_dict:
    print("Parsed profile:", profile_dict)
else:
    print("Could not extract dictionary.")



 Assistant Response:

{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}
Parsed profile: {
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}


### iterate_response() - Helper Function:
We've created a small helper test function to ensure the model's response is consistent.
Uncomment the code blocks and run the function `iterate_response(response)` to check if the response of the `intent_confirmation_layer`is consistent.}

In [28]:
def iterate_llm_response(funct, debug_response, num = 10):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer'
# iterate_llm_response(get_chat_completions, messages)

In [29]:
profile_dict

'{\n  "interests": ["culture", "food"],\n  "duration_days": 2,\n  "budget_aed": 600,\n  "group_type": "couple"\n}'

In [30]:
# Check for LLM function's consistency
iterate_llm_response(dictionary_present, profile_dict)


 Assistant Response:

{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}
Iteration: 0
{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}
--------------------------------------------------

 Assistant Response:

{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}
Iteration: 1
{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}
--------------------------------------------------

 Assistant Response:

{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}
Iteration: 2
{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budget_aed": 600,
  "group_type": "couple"
}
--------------------------------------------------

 Assistant Response:

{
  "interests": ["culture", "food"],
  "duration_days": 2,
  "budge

In [31]:
def product_mapping(user_profile, products=dubai_tourist_spots):
    """
    Maps user profile to matching Dubai products.

    Args:
        user_profile (dict): Extracted user info with keys:
            'interests' (list[str]), 'budget_aed' (int),
            'duration_days' (int), 'group_type' (str)

        products (list[dict]): Catalog of products/experiences

    Returns:
        list[dict]: Matching products sorted by relevance
    """

    matches = []

    user_interests = set(i.lower() for i in user_profile.get("interests", []))
    budget = user_profile.get("budget_aed", 0)
    group = user_profile.get("group_type", "").lower()

    for product in products:
        tags = set(t.lower() for t in product["tags"])
        if user_interests.intersection(tags):
            # Check budget fit
            if product["min_budget"] <= budget <= product["max_budget"]:
                # Check group suitability
                if group in [g.lower() for g in product["suitable_for"]]:
                    matches.append(product)

    # Optional: Sort by how many tags match (descending)
    matches.sort(key=lambda p: len(user_interests.intersection(set(t.lower() for t in p["tags"]))), reverse=True)

    return matches


In [106]:
def parse_columns(df):
    for col in ["tags", "suitable_for"]:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

In [98]:
def extract_information(raw_text):
    """
    Uses LLM to extract structured info (interests, budget, duration, group_type)
    from raw user input.
    """
    messages = [
                {"role": "system", "content": "You are an assistant that extracts structured travel preferences from user input. Return a Python dictionary."},
                {"role": "user", "content": f"""Extract the following from the user's message as a Python dictionary:
                - interests (list of strings)
                - duration_days (integer)
                - budget_aed (integer)
                - group_type (string like 'family', 'couple', 'solo', or 'friends')
                
                User Input: \"\"\"{raw_text}\"\"\""""},
            ]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.2,
            max_tokens=200
        )
        response_text = response.choices[0].message.content.strip()

        # Safe eval (only if you're confident in the format being returned)
        parsed_dict = ast.literal_eval(response_text)
        if isinstance(parsed_dict, dict):
            return parsed_dict
        else:
            return None

    except Exception as e:
        print(f"Error extracting information: {e}")
        return None


In [110]:
# Raw user input
user_input = "We're a couple visiting Dubai for 2 days. We love culture and food with a budget of 600 AED."

# Step 1: Extract info
profile = extract_information(user_input)
print("Extracted profile:", profile)

# Step 2: Map products
if profile:
    recommended = product_mapping(profile, dubai_tourist_spots.to_dict(orient="records"))
    print("\nRecommended Experiences:")
    for p in recommended:
        print(f"- {p['name']}: {p['description']}")
else:
    print("Could not extract user profile.")


Extracted profile: {'interests': ['culture', 'food'], 'duration_days': 2, 'budget_aed': 600, 'group_type': 'couple'}

Recommended Experiences:


In [34]:
def match_experiences_to_profile(user_profile, experiences=dubai_tourist_spots):
    """
    Match Dubai experiences to user profile with scoring and detailed reasoning.

    Args:
        user_profile (dict): Dictionary with keys:
            - interests (list[str])
            - budget_aed (int)
            - duration_days (int)
            - group_type (str)

        experiences (list[dict]): List of Dubai experiences.

    Returns:
        list of dict: Sorted recommended experiences with scores and reasons.
    """

    # Convert list of dicts into DataFrame if needed
    if not isinstance(experiences, pd.DataFrame):
        experiences = pd.DataFrame(experiences)

    # Required columns
    expected = {"name", "tags", "min_budget", "max_budget", 
                "duration_hours", "suitable_for", "description"}
    if not expected.issubset(experiences.columns):
        missing = expected - set(experiences.columns)
        raise KeyError(f"Missing required columns: {missing}")

    # Preprocess columns: lowercase tags & suitable_for lists
    df = experiences.assign(
        tags=experiences["tags"].map(lambda tags: [t.lower().strip() for t in tags]),
        suitable_for=experiences["suitable_for"].map(lambda grp: [g.lower().strip() for g in grp])
    )

    user_interests = set(i.lower().strip() for i in user_profile.get("interests", []))
    budget = user_profile.get("budget_aed", 0)
    group_type = user_profile.get("group_type", "").lower()
    duration_days = float(user_profile.get("duration_days", 0))
    available_hours = duration_days * 8

    def _compute_score(row):
        score = 0
        nodes = []

        # Interest overlap → 10 pts per matched tag (max 40)
        overlap = user_interests.intersection(set(row["tags"]))
        i_score = min(len(overlap) * 10, 40)
        score += i_score
        if overlap:
            nodes.append(f"Interest match: {', '.join(sorted(overlap))} +{i_score}")

        # Budget fit → 30 points if within range, else 0
        if row["min_budget"] <= budget <= row["max_budget"]:
            score += 30
            nodes.append(f"Budget {budget} within [{row['min_budget']}-{row['max_budget']}] +30")
        else:
            nodes.append(f"Budget {budget} outside range +0")

        # Duration check → 20 points if fits
        if row["duration_hours"] <= available_hours:
            score += 20
            nodes.append(f"Duration {row['duration_hours']}h fits in {available_hours}h +20")
        else:
            nodes.append(f"Duration {row['duration_hours']}h too long +0")

        # Group suitability → 10 points if group type matches
        if group_type in row["suitable_for"]:
            score += 10
            nodes.append(f"Group '{group_type}' is suitable +10")
        else:
            nodes.append(f"Group '{group_type}' not a match +0")

        return pd.Series({"score": score, "rationale": "; ".join(nodes)})

    # Apply scoring
    scored = df.apply(_compute_score, axis=1)
    df = df.join(scored)

    # Sort by score desc and return as list of dicts
    df_sorted = df.sort_values(by="score", ascending=False)

    return df_sorted[["name", "description", "score", "rationale"]].to_dict(orient="records")


In [35]:
user_profile = {
    "interests": ["culture", "shopping", "beach"],
    "budget_aed": 600,
    "duration_days": 2,
    "group_type": "family"
}

recommendations = match_experiences_to_profile(user_profile, dubai_tourist_spots)


In [36]:
from typing import List, Dict, Any

def product_recommendation_layer(
    scored_recs: List[Dict[str, Any]],
    top_n: int = 3
) -> str:
    """
    Select and format top-N recommended experiences for user output.

    Parameters:
      - scored_recs (List[dict]): List of experiences sorted descending by 'score',
        where each dict has keys: 'name', 'description', 'score', 'rationale'.
      - top_n (int): Number of top items to include in the final message.

    Returns:
      - message (str): A polished conversational recommendation prompt.
    """

    if not scored_recs:
        return "Sorry, I couldn't find any experiences that matched your preferences. Could you share more details?"

    # Safeguard top_n
    top_n = min(top_n, len(scored_recs))

    lines = [
        f"Here are the top {top_n} experiences I recommend:"
    ]

    for idx, rec in enumerate(scored_recs[:top_n], start=1):
        lines.append(f"\n{idx}. **{rec['name']}**")
        lines.append(f"   • Description: {rec['description']}")
        lines.append(f"   • Score: {rec['score']} points")
        lines.append(f"   • Why I picked it: {rec['rationale']}")

    lines.append("\nWould you like to book one of these, or explore more options?")

    return "\n".join(lines)


In [37]:
def initialize_conv_reco(user_profile):
    """
    Initialize conversation with product recommendations based on user profile.

    Args:
        user_profile (dict): Extracted user preferences dictionary

    Returns:
        str: Assistant message with product recommendations
    """

    # Step 1: Match experiences
    recommendations = match_experiences_to_profile(user_profile)

    # Step 2: Generate recommendation message
    recommendation_message = product_recommendation_layer(recommendations)

    # You can add additional personalization or conversation starter text here

    return recommendation_message


In [38]:
user_profile = {
    "interests": ["culture", "shopping", "beach"],
    "budget_aed": 600,
    "duration_days": 2,
    "group_type": "family"
}

assistant_message = initialize_conv_reco(user_profile)
print(assistant_message)


Here are the top 3 experiences I recommend:

1. **Desert Safari**
   • Description: Dune‑bashing, camel rides, traditional dinner under the stars.
   • Score: 50 points
   • Why I picked it: Budget 600 within [300-700] +30; Duration 6.0h fits in 16.0h +20; Group 'family' not a match +0

2. **Desert Safari**
   • Description: Dune‑bashing, camel rides, traditional dinner under the stars.
   • Score: 50 points
   • Why I picked it: Budget 600 within [300-700] +30; Duration 6.0h fits in 16.0h +20; Group 'family' not a match +0

3. **Desert Safari**
   • Description: Dune‑bashing, camel rides, traditional dinner under the stars.
   • Score: 50 points
   • Why I picked it: Budget 600 within [300-700] +30; Duration 6.0h fits in 16.0h +20; Group 'family' not a match +0

Would you like to book one of these, or explore more options?


In [91]:
def extract_dictionary_from_string(text):
    """
    Attempts to find and parse the first valid dictionary in a string.
    """
    try:
        match = re.search(r"\{(?:[^{}]|(?R))*\}", text, re.DOTALL)
        if not match:
            return None
        dict_str = match.group(0)
        return ast.literal_eval(dict_str)
    except Exception as e:
        print("Dictionary parsing error:", e)
        return None

In [39]:
def clarify_prompt(missing_keys):
    prompts = []
    if "interests" in missing_keys:
        prompts.append("What types of experiences interest you? (e.g., culture, beach, food)")
    if "budget_aed" in missing_keys:
        prompts.append("What is your budget in AED?")
    if "duration_days" in missing_keys:
        prompts.append("How many days will you spend in Dubai?")
    if "group_type" in missing_keys:
        prompts.append("Are you traveling alone, as a couple, with family, or friends?")
    return "I just need a few more details: " + " ".join(prompts)


In [40]:
class DialogueManagementSystem:
    def __init__(self):
        self.state = "INIT"
        self.profile = {}
        self.last_recommendations = []

    def dialogue_mgmt_system(self, user_msg: str) -> str:
        # Step 1: Moderate unsafe content
        if moderation_check(user_msg):
            return "Sorry, that message isn't allowed. Please rephrase."

        # Step 2: INIT state
        if self.state == "INIT":
            greeting = initialize_conversation()

            # Step 2a: Check if user already gave dictionary-like info
            if dictionary_present(user_msg):
                user_dict = extract_dictionary_from_string(user_msg)
                confirmation = intent_confirmation_layer(user_dict)
                self.profile = user_dict
                self.state = "PROFILE_CONFIRMED"
                product_mapping_layer(self.profile)  # Optional: categorization before match
                recs = match_experiences_to_profile(self.profile)
                self.last_recommendations = recs
                return greeting + "\n\n" + confirmation + "\n" + product_recommendation_layer(recs)

            # Step 2b: Extract from free-form input
            info = extract_information(user_msg)
            if not info or info.get("missing_info"):
                self.profile = info
                self.state = "PROFILE_COLLECTING"
                return greeting + "\n\n" + clarify_prompt(info.get("missing_info", []))

            self.profile = info
            self.state = "PROFILE_CONFIRMED"
            confirmation = intent_confirmation_layer(self.profile)
            product_mapping_layer(self.profile)
            recs = match_experiences_to_profile(self.profile)
            self.last_recommendations = recs
            return greeting + "\n\n" + confirmation + "\n" + product_recommendation_layer(recs)

        # Step 3: Fill missing info
        elif self.state == "PROFILE_COLLECTING":
            info = extract_information(user_msg)
            if info:
                self.profile.update({k: v for k, v in info.items() if v is not None})

            if self.profile.get("missing_info"):
                return clarify_prompt(self.profile["missing_info"])

            self.state = "PROFILE_CONFIRMED"
            confirmation = intent_confirmation_layer(self.profile)
            product_mapping_layer(self.profile)
            recs = match_experiences_to_profile(self.profile)
            self.last_recommendations = recs
            return confirmation + "\n" + product_recommendation_layer(recs)

        # Step 4: After confirmation, allow booking or changes
        elif self.state == "PROFILE_CONFIRMED":
            msg = user_msg.lower()
            if "book" in msg or "reserve" in msg:
                self.state = "BOOKING_REQUEST"
                return "Great! Which experience would you like to book? Type the exact name."
            if "change" in msg or "update" in msg:
                self.state = "PROFILE_COLLECTING"
                self.profile["missing_info"] = ["interests", "budget_aed", "duration_days", "group_type"]
                return "Sure, what would you like to change?"

            return "Would you like to book one of these, or update your preferences?"

        # Step 5: Booking
        elif self.state == "BOOKING_REQUEST":
            selected = user_msg.lower()
            matched = next((e for e in self.last_recommendations if e["experience"].lower() == selected), None)
            if matched:
                self.state = "BOOKING_CONFIRMED"
                return f"Booking confirmed for **{matched['experience']}**! Have an amazing experience."
            return "Couldn't find that experience in your list. Please copy the name exactly."

        # Step 6: Booking confirmed
        elif self.state == "BOOKING_CONFIRMED":
            self.state = "INIT"
            self.profile = {}
            return "Booking successful. Would you like to plan something else?"

        # Fallback
        else:
            self.state = "INIT"
            return "Something went wrong. Let's start over. Tell me your preferences!"


In [41]:
def evaluate_model_response(tagged_dict, model_dict):
    """
    Compare the reference (tagged) user profile dict with the model's extracted profile dict.
    Returns evaluation metrics and mismatch details.
    """

    metrics = {"total_fields": 0, "correct": 0, "incorrect": []}

    for key in ["interests", "budget_aed", "duration_days", "group_type"]:
        metrics["total_fields"] += 1
        ref = tagged_dict.get(key)
        pred = model_dict.get(key)

        if ref == pred:
            metrics["correct"] += 1
        else:
            metrics["incorrect"].append({"field": key, "expected": ref, "predicted": pred})

    accuracy = metrics["correct"] / metrics["total_fields"]
    return {
        "accuracy": accuracy,
        "correct_fields": metrics["correct"],
        "total_fields": metrics["total_fields"],
        "mismatches": metrics["incorrect"]
    }


In [42]:
tagged = {
    "interests": ["culture", "food"], 
    "budget_aed": 600, 
    "duration_days": 2, 
    "group_type": "family"
}
model = {
    "interests": ["culture", "shopping"], 
    "budget_aed": 600, 
    "duration_days": 2, 
    "group_type": "family"
}

result = evaluate_model_response(tagged, model)
print(result)

{'accuracy': 0.75, 'correct_fields': 3, 'total_fields': 4, 'mismatches': [{'field': 'interests', 'expected': ['culture', 'food'], 'predicted': ['culture', 'shopping']}]}


### Future Scope of Work for Dubai Local Explorer AI Chatbot

1. **Context-Aware Memory & Session Persistence**

    ***Objective:*** Implement a memory module to retain user profiles, preferences, and past interactions across sessions.

    ***Implementation:***

    - Utilize technologies like Redis or vector databases (e.g., Pinecone, FAISS) for session storage.
    
    - Ensure secure handling of user data, complying with GDPR and local data protection regulations.
    
    - Provide users with options to view, update, or delete their stored information.

2. **Live Integration with Real-Time APIs**

    ***Objective:*** Enhance the chatbot's recommendations with up-to-date information.

    ***Implementation:***

    - Integrate with Dubai's tourism APIs to fetch real-time data on attractions, events, and experiences.
    
    - Incorporate weather APIs to adjust recommendations based on current conditions.
    
    - Connect with booking platforms to provide users with seamless reservation options.

3. ***Feedback Learning Loop***

    ***Objective:*** Enable the chatbot to learn from user feedback and improve over time.

    ***Implementation:***
    
    - Implement thumbs-up/thumbs-down feedback mechanisms after each recommendation.
    
    - Analyze feedback to identify areas for improvement in the chatbot's responses.
    
    - Use insights to fine-tune the chatbot's algorithms and enhance user satisfaction.

4. **Natural Language Understanding (NLU) Enhancements**

    ***Objective:*** Improve the chatbot's understanding of user inputs.
    
    ***Implementation:***
    
    - Fine-tune NLU models to better handle diverse user queries and intents.
    
    - Implement multi-intent detection to process complex user requests.
    
    - Enhance disambiguation capabilities to clarify user inputs when necessary.

5. **Multilingual Support**

    ***Objective:*** Cater to Dubai's diverse population by supporting multiple languages.
    
    ***Implementation:***
    
    - Integrate translation APIs to support languages such as Arabic, Hindi, Urdu, and Chinese.
    
    - Ensure seamless language switching within conversations.
    
    - Adapt cultural nuances and idiomatic expressions for each supported language.

6. **Smart Itinerary Generator**

    ***Objective:*** Provide users with personalized travel itineraries.
    
    ***Implementation:***
    
    - Develop algorithms to generate itineraries based on user preferences, budget, and duration of stay.
    
    - Incorporate real-time data to adjust itineraries dynamically.
    
    - Allow users to customize and save their itineraries for future reference.

7. **Voice + WhatsApp Integration**

    ***Objective:*** Expand the chatbot's reach through voice and messaging platforms.
    
    ***Implementation:***
    
    - Integrate with voice recognition APIs to support voice inputs.
    
    - Connect with WhatsApp Business API to facilitate conversations via messaging.
    
    - Ensure consistent user experience across different platforms.

8. **User Authentication & Dashboard**

    ***Objective:*** Provide users with personalized experiences and control over their data.
    
    ***Implementation:***
    
    - Implement user authentication using OAuth or JWT tokens.
    
    - Develop a dashboard for users to manage their profiles, preferences, and saved itineraries.
    
    - Ensure data privacy and security in line with best practices.

9. **AI Reasoning Evaluation Suite**

    ***Objective:*** Continuously assess and improve the chatbot's reasoning capabilities.
    
    ***Implementation:***
    
    - Develop evaluation metrics to assess the accuracy and relevance of the chatbot's responses.
    
    - Implement logging and monitoring tools to track performance over time.
    
    - Use insights to refine algorithms and enhance decision-making processes.

10. **Scalability and Deployment**

    ***Objective:*** Ensure the chatbot can handle increased user traffic and data.
    
    ***Implementation:***
    
    - Containerize the application using Docker for consistent deployment environments.
    
    - Deploy using Kubernetes to manage scaling and orchestration.
    
    - Implement load balancing and auto-scaling to handle varying traffic loads.

11. **UX & UI Enhancements**

    ***Objective:*** Improve the user interface and experience.
    
    ***Implementation:***
    
    - Redesign the chatbot interface to be more intuitive and user-friendly.
    
    - Implement features like typing indicators, quick replies, and rich media support.
    
    - Ensure responsiveness across different devices and screen sizes.

12. **Security, Moderation & Data Privacy**

    ***Objective:*** Protect user data and ensure compliance with regulations.
    
    ***Implementation:***
    
    - Implement data encryption for all user interactions.
    
    - Integrate moderation tools to filter inappropriate content.
    
    - Regularly audit and update security protocols to address emerging threats.

**Additional Considerations**

- **Ethical AI Practices:** Adhere to Dubai's AI Ethics Guidelines to ensure transparency, fairness, and accountability in AI-driven decisions.

- **User-Centric Design:** Continuously gather user feedback to inform iterative improvements and ensure the chatbot meets user needs and expectations.

- **Collaboration with Local Entities:** Partner with local tourism boards, businesses, and government agencies to provide users with accurate and comprehensive information.